In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np
import datetime
import copy
import seaborn as sns
import scipy
import psycopg2
from dataclasses import dataclass

In [ ]:
username = input("Username: ")
password = input("Password: ")

# Clear output
from IPython.display import clear_output

clear_output()

In [ ]:
conn = psycopg2.connect(
    user=username,
    password=password,
    host="portal-production-postgres.c1trxszive18.us-west-2.rds.amazonaws.com",
    port="5432",
    database="portal_production_postgres",
)

cursor = conn.cursor()

# Z Test

In [ ]:
from dataclasses import dataclass
import numpy as np
import scipy.stats
from math import sqrt


@dataclass
class ProportionTestSample:
    false_positive_rate: float
    n: int


@dataclass
class ZTestResult:
    p_value: float
    z_score: float


def two_tailed_test_z_score_to_p_value(z_score: float) -> float:
    return scipy.stats.norm.sf(abs(z_score)) * 2


def two_proportion_z_test(
    observed_sample: ProportionTestSample, merged_sample: ProportionTestSample
):
    p1_hat = observed_sample.false_positive_rate
    p2_hat = merged_sample.false_positive_rate

    n1 = observed_sample.n
    n2 = merged_sample.n

    p_hat = (p1_hat * n1 + p2_hat * n2) / (n1 + n2)
    z_score = (p1_hat - p2_hat) / sqrt(p_hat * (1 - p_hat) * (1 / n1 + 1 / n2))
    return ZTestResult(
        p_value=two_tailed_test_z_score_to_p_value(z_score), z_score=z_score
    )

# Extracting Delorean Incidents

In [ ]:
from enum import Enum, unique


@unique
class ReviewType(Enum):
    DEFAULT_UNKNOWN = 0
    TRUE_POSITIVE = 1
    FALSE_POSITIVE = 2
    UNSURE = 3


def positive_negative_to_type(valid_count, invalid_count, unsure_count):
    if valid_count > 0 and invalid_count == 0:
        return ReviewType.TRUE_POSITIVE
    if invalid_count > 0 and valid_count == 0:
        return ReviewType.FALSE_POSITIVE
    return ReviewType.UNSURE


def get_additional_row_data(incidents_from_portal: pd.DataFrame):
    """Make track_uuid, sequence_id, and run_uuid accessible"""
    incidents_from_portal["track_uuid"] = incidents_from_portal.data.apply(
        lambda data: data["track_uuid"]
    )
    incidents_from_portal["sequence_id"] = incidents_from_portal.data.apply(
        lambda data: data.get("sequence_id", "None")
    )
    incidents_from_portal["run_uuid"] = incidents_from_portal.data.apply(
        lambda data: data.get("run_uuid", "None")
    )
    incidents_from_portal["incident_type"] = incidents_from_portal.data.apply(
        lambda data: data["incident_type_id"]
    )

In [ ]:
# Delorean Incidents
sql = (
    "SELECT * "
    "FROM api_incident "
    "WHERE data->>'cooldown_tag' = 'True' "
    "AND data->>'track_uuid' is not null "
    "AND "
    "((data->>'incident_type_id' in "
    "('PRODUCTION_LINE_DOWN', 'SPILL', 'OPEN_DOOR_DURATION', 'N_PERSON_PED_ZONE') "
    "AND data->>'run_uuid' is not null) "
    "OR data->>'incident_type_id' not in "
    "('PRODUCTION_LINE_DOWN', 'SPILL', 'OPEN_DOOR_DURATION', 'N_PERSON_PED_ZONE')) "
    "ORDER BY data->>'track_uuid', created_at"
)

delorean_incidents = pd.read_sql(sql, conn)
delorean_incidents["review_category"] = delorean_incidents.apply(
    lambda row: positive_negative_to_type(
        row["valid_feedback_count"],
        row["invalid_feedback_count"],
        row["unsure_feedback_count"],
    ),
    axis=1,
)
get_additional_row_data(delorean_incidents)
print(f"DELOREAN INCIDENTS SHAPE: {delorean_incidents.shape}")

# Head Incidents
track_uuids = tuple(delorean_incidents.track_uuid.unique())

sql = (
    "SELECT * "
    "FROM api_incident "
    f"WHERE data->>'track_uuid' in {track_uuids} "
    "AND data->>'cooldown_tag' = 'False' "
    "AND "
    "((data->>'incident_type_id' in "
    "('PRODUCTION_LINE_DOWN', 'SPILL', 'OPEN_DOOR_DURATION', 'N_PERSON_PED_ZONE') "
    "AND data->>'run_uuid' is not null) "
    "OR data->>'incident_type_id' not in "
    "('PRODUCTION_LINE_DOWN', 'SPILL', 'OPEN_DOOR_DURATION', 'N_PERSON_PED_ZONE')) "
    "ORDER BY data->>'track_uuid', created_at"
)
head_incidents = pd.read_sql(sql, conn)
head_incidents["review_category"] = head_incidents.apply(
    lambda row: positive_negative_to_type(
        row["valid_feedback_count"],
        row["invalid_feedback_count"],
        row["unsure_feedback_count"],
    ),
    axis=1,
)
get_additional_row_data(head_incidents)
print(f"PRIMARY INCIDENTS SHAPE: {head_incidents.shape}")

In [ ]:
import plotly.express as px

incident_order = {
    "title": [
        "BAD_POSTURE",
        "DOOR_VIOLATION",
        "HARD_HAT",
        "NO_PED_ZONE",
        "NO_STOP_AT_DOOR_INTERSECTION",
        "NO_STOP_AT_END_OF_AISLE",
        "NO_STOP_AT_INTERSECTION",
        "N_PERSON_PED_ZONE",
        "OPEN_DOOR_DURATION",
        "OVERREACHING",
        "PARKING_DURATION",
        "PIGGYBACK",
        "PRODUCTION_LINE_DOWN",
        "SAFETY_VEST",
        "SPILL",
    ]
}

fig = px.histogram(
    head_incidents,
    x="title",
    color="title",
    barmode="group",
    height=400,
    labels={
        "title": "Incident Type",
    },
    category_orders=incident_order,
    title="Head Incidents",
)
fig.show()

fig2 = px.histogram(
    delorean_incidents,
    x="title",
    color="title",
    barmode="group",
    height=400,
    labels={
        "title": "Incident Type",
    },
    category_orders=incident_order,
    title="Delorean Incidents",
)
fig2.show()

# Head Merging Analysis

In [ ]:
def calculate_merge_percentage(
    first_incidents: pd.DataFrame, tail_incidents: pd.DataFrame
):
    incident_types = []
    count_head_incidents = []
    count_head_incidents_with_tail = []
    count_tail_incidents = []
    count_tp_head = []
    count_fp_head = []
    count_uk_head = []
    tp_predictive_ratios = []
    fp_predictive_ratios = []
    uk_predictive_ratios = []
    for incident_type in np.sort(first_incidents.incident_type.unique()):
        # Get all incidents of type incident_type
        typed_head_incidents = first_incidents[
            first_incidents.incident_type == incident_type
        ]
        has_tail = []
        typed_tail_incidents = tail_incidents[
            tail_incidents.incident_type == incident_type
        ]
        count_no_tail = 0
        count_tail_tp_given_head_tp = 0
        count_tail_given_head_tp = 0
        count_tail_fp_given_head_fp = 0
        count_tail_given_head_fp = 0
        count_tail_uk_given_head_uk = 0
        count_tail_given_head_uk = 0
        for _, head_incident in typed_head_incidents.iterrows():
            track_uuid = head_incident.track_uuid
            # Get groupings for static actor incidents
            if incident_type in (
                "PRODUCTION_LINE_DOWN",
                "SPILL",
                "OPEN_DOOR_DURATION",
                "N_PERSON_PED_ZONE",
            ):
                run_uuid = head_incident.run_uuid
                sequence_id = head_incident.sequence_id
                subsequent_incidents = typed_tail_incidents[
                    (typed_tail_incidents.track_uuid == track_uuid)
                    & (typed_tail_incidents.run_uuid == run_uuid)
                    & (typed_tail_incidents.sequence_id == sequence_id)
                ]
            # Get groupings for non static actor incidents
            else:
                subsequent_incidents = typed_tail_incidents[
                    typed_tail_incidents.track_uuid == track_uuid
                ]

            if subsequent_incidents.empty:
                count_no_tail += 1
                has_tail.append(False)
                continue

            has_tail.append(True)
            head_review_type = head_incident.review_category
            tail_head_match = subsequent_incidents.review_category.eq(
                head_review_type
            )
            if head_review_type == ReviewType.TRUE_POSITIVE:
                count_tail_tp_given_head_tp += tail_head_match.sum()
                count_tail_given_head_tp += tail_head_match.size
            elif head_review_type == ReviewType.FALSE_POSITIVE:
                count_tail_fp_given_head_fp += tail_head_match.sum()
                count_tail_given_head_fp += tail_head_match.size
            else:
                count_tail_uk_given_head_uk += tail_head_match.sum()
                count_tail_given_head_uk += tail_head_match.size

        incident_types.append(incident_type)
        count_head_incidents.append(typed_head_incidents.shape[0])
        count_head_incidents_with_tail.append(
            typed_head_incidents.shape[0] - count_no_tail
        )
        count_tp_head.append(
            typed_head_incidents[has_tail]
            .review_category.eq(ReviewType.TRUE_POSITIVE)
            .sum()
        )
        count_fp_head.append(
            typed_head_incidents[has_tail]
            .review_category.eq(ReviewType.FALSE_POSITIVE)
            .sum()
        )
        count_uk_head.append(
            typed_head_incidents[has_tail]
            .review_category.eq(ReviewType.UNSURE)
            .sum()
        )
        count_tail_incidents.append(typed_tail_incidents.shape[0])
        tp_predictive_ratios.append(
            float(count_tail_tp_given_head_tp) / count_tail_given_head_tp
            if count_tail_given_head_tp != 0
            else None
        )
        fp_predictive_ratios.append(
            float(count_tail_fp_given_head_fp) / count_tail_given_head_fp
            if count_tail_given_head_fp != 0
            else None
        )
        uk_predictive_ratios.append(
            float(count_tail_uk_given_head_uk) / count_tail_given_head_uk
            if count_tail_given_head_uk != 0
            else None
        )
    incident_merge_percentage = {
        "incident_type": incident_types,
        "count_head_incidents": count_head_incidents,
        "count_head_incidents_with_tail": count_head_incidents_with_tail,
        "count_tail_incidents": count_tail_incidents,
        "count_tp_head": count_tp_head,
        "count_fp_head": count_fp_head,
        "count_uk_head": count_uk_head,
        "tp_predictive_ratios": tp_predictive_ratios,
        "fp_predictive_ratios": fp_predictive_ratios,
        "uk_predictive_ratios": uk_predictive_ratios,
    }
    return incident_merge_percentage

In [ ]:
x = pd.DataFrame(
    calculate_merge_percentage(head_incidents, delorean_incidents)
)
display(x)